In [562]:
environment = [] # array of models that will interact

In [561]:
def model_wrapper(model):
    return {
        'model':model,
        'current_tokens': 10
    }

In [563]:
for i in range(5):
    environment.append(model_wrapper(i))

print(environment)

[{'model': 0, 'current_tokens': 10}, {'model': 1, 'current_tokens': 10}, {'model': 2, 'current_tokens': 10}, {'model': 3, 'current_tokens': 10}, {'model': 4, 'current_tokens': 10}]


In [299]:
'''
    the agent with the highest number of tokens is the one chosen
'''
def max_strategy(environment, price_to_pay):
    max_index = 0

    for i in range(1 ,len(environment)):
        val = environment[i]['current_tokens']
        if val > environment[max_index]['current_tokens']:
            max_index = i

    assert price_to_pay <= environment[max_index]['current_tokens']
    environment_output = environment.copy()
    environment_output[max_index]['current_tokens']-= price_to_pay
    print("choosen" , max_index)
    return environment_output

In [300]:
'''
    Instead of having a price to pay, it will win the one that bids the most
    The one that has more tokens has the most likelihood to win the current round
'''
def auction_max_random_strategy(environment, price_to_pay):
    max_index = 0
    max_bid = 0

    for i in range(len(environment)):
        current_tokens = environment[i]['current_tokens']
        # random part
        to_bid = random.randint(0, current_tokens + 1)
        # max part
        if max_bid <= to_bid:
            max_bid = to_bid
            max_index = i

    environment_output = environment.copy()
    environment_output[max_index]['current_tokens']-= max_bid

    print("o mano ", max_index, " pagou ", max_bid)

    return environment_output




In [301]:
import random
import numpy

'''
    chooses a random agent to continue the play
'''
def random_strategy(environment, price_to_pay):
    print(random.choice(environment))
    environment_output = environment.copy()
    environment_output[numpy.random.randint(0, len(environment) - 1)]['current_tokens'] -= price_to_pay

    return environment_output



In [308]:
'''
    wins the auction the first one to bid
    It starts with a high price to pay and that number is reduced
    The probability to accept a certain price is going to be dependent on the current number of tokens
'''
def dutch_auction(environment, price_to_pay):
    accepted_index = -1
    reducing_price = 1

    current_price_to_pay = price_to_pay

    while accepted_index == -1:
        print(current_price_to_pay)

        for i in range(len(environment)):
            val = environment[i]['current_tokens']
            if current_price_to_pay > val:
                continue

            percentage_gotten = (val - current_price_to_pay)/val 

            threshold = random.uniform(0, 1)

            #print(percentage_gotten, threshold)

            # this condition is always met
            if percentage_gotten >= threshold:
                accepted_index = i
                break

        current_price_to_pay = max(current_price_to_pay - reducing_price, 0)

    
    environment_output = environment.copy()
    if accepted_index == -1:
        accepted_index = random.randint(0, len(environment)- 1)
    
    environment_output[accepted_index]['current_tokens'] -= current_price_to_pay

    #print("aceitou no dutch o ", accepted_index)
    return environment_output

In [303]:
STATE_VALUE = 5

In [555]:
'''
    changes the environment state
    it should guarantee that there is always token available to bid
'''
def advance_state(environment, extra = 1):

    environment_output = environment.copy()
    for i in range(len(environment)):
        environment_output[i]['current_tokens']+= STATE_VALUE * extra

    return environment_output
    

In [305]:
def choose_agent(environment, strategy_callback):
    price_to_pay = STATE_VALUE
    new_environment = strategy_callback(environment, price_to_pay)
    return new_environment
    

In [ ]:
NUMBER_OF_ROUNDS = 10

print(environment)
for i in range(NUMBER_OF_ROUNDS):
    environment = choose_agent(environment, dutch_auction)
    environment = advance_state(environment)
    

In [307]:
for x in environment:
    print(x)

{'model': 0, 'current_tokens': 32}
{'model': 1, 'current_tokens': 52}
{'model': 2, 'current_tokens': 56}
{'model': 3, 'current_tokens': 60}
{'model': 4, 'current_tokens': 60}


# Different stuff


In [552]:
from collections import defaultdict
'''
    the auction is going to be done for the next five products
    The price that each agent is going to give will be:
        * For one isolated element randint(0, current_price_to_pay / k)
        * For k1 consecutive elements randint(0, current_price_to_pay/k) * K1 * float_random(1, 1.5) => valoriza
        * For an arbirtrary sequence calculate separetly the value for those consecutive sequence and then sum them
    Should this be the vcg?
'''
K = 5

def create_values_for_agent(agent, price_to_pay):

    # notes together values
    note_together_values = defaultdict()
    one_note = int(price_to_pay/K + 1)
    note_together_values[1] = one_note
    for i in range(2, K + 1):
        note_together_values[i] = one_note * i * random.uniform(1,1.1)

    return note_together_values



'''
    gets the option that is more lucrative when everything is sold
'''
def get_best_option_non_seq(environment_wrapper, size, current_value, result):

    if size == 0:
        return current_value, result

    
    value = - 1
    current_one = []
    for agent in environment_wrapper:
        for element_size, option in agent['valorization'].items():
            if size - element_size >= 0:
                value, one = get_best_option_non_seq(environment_wrapper, size - element_size, current_value + option, \
                    result + [{'model': agent['model'], 'value': option}])
                
                if value > current_value:
                    current_one = one
                    current = value

    
    return value, current_one
            

'''
    attribute the next k elements to the agents according to the one that maximizes the value
'''
def auction_next_k(environment, price_to_pay):
    import copy
    environment_output = copy.deepcopy(environment)

    for agent in environment:
        vals = create_values_for_agent(agent, price_to_pay)
        agent['valorization'] = vals # prices they are willing to pay for N products

    print("here", environment_output)

    
    _, result = get_best_option_non_seq(environment, K, 0, [])

    for element in result:
        for agent in environment_output:
            if agent['model'] == element['model']:
                agent['current_tokens'] -= element['value']
    

    return environment_output
    #for agent in environment:
    #    print(agent)

In [557]:
NUMBER_OF_ROUNDS = 10

for i in range(int(NUMBER_OF_ROUNDS/K)):
    environment = auction_next_k(environment, STATE_VALUE)
    environment = advance_state(environment, K)

here [{'model': 0, 'current_tokens': 32, 'valorization': defaultdict(None, {1: 2, 2: 4.159177549566412, 3: 6.442255982740718, 4: 8.49133188157037, 5: 10.798710456928827})}, {'model': 1, 'current_tokens': 52, 'valorization': defaultdict(None, {1: 2, 2: 4.214785714389888, 3: 6.154601198563395, 4: 8.768398928401288, 5: 10.658256357149527})}, {'model': 2, 'current_tokens': 56, 'valorization': defaultdict(None, {1: 2, 2: 4.112820582536904, 3: 6.579633257441216, 4: 8.311780539887712, 5: 10.853696562017792})}, {'model': 3, 'current_tokens': 60, 'valorization': defaultdict(None, {1: 2, 2: 4.030841730687423, 3: 6.265411833181592, 4: 8.543971204882169, 5: 10.004869052044448})}, {'model': 4, 'current_tokens': -14.360658468719878, 'valorization': defaultdict(None, {1: 2, 2: 4.034104165365007, 3: 6.545070608843026, 4: 8.161153984872774, 5: 10.387056237877225})}]
here [{'model': 0, 'current_tokens': 57, 'valorization': defaultdict(None, {1: 2, 2: 4.159177549566412, 3: 6.442255982740718, 4: 8.4913318

# Another approach


In [572]:
def similarity_measure(chord1, chord2):
    return random.randint(1,10)

In [560]:
def next_note_predict(model):
    return 1

In [631]:
'''
    defines a rank of preferences for each agent in relation to other
'''
def preferences(environment):
    
    preferences = defaultdict(defaultdict)
    for agent in environment:
        for other_agent in environment:
            if other_agent != agent:
                preferences[agent['model']][other_agent['model']] = similarity_measure(next_note_predict(agent), next_note_predict(other_agent))

    return preferences

preferences = preferences(environment)

In [632]:
len(preferences), preferences

(5,
 defaultdict(collections.defaultdict,
             {0: defaultdict(None, {1: 1, 2: 10, 3: 4, 4: 7}),
              1: defaultdict(None, {0: 9, 2: 9, 3: 7, 4: 10}),
              2: defaultdict(None, {0: 7, 1: 5, 3: 7, 4: 4}),
              3: defaultdict(None, {0: 2, 1: 5, 2: 2, 4: 8}),
              4: defaultdict(None, {0: 4, 1: 3, 2: 3, 3: 7})}))

In [670]:
def vote_counters(c1, c2, preferences):

    c1_votes = 0
    c2_votes = 0

    for candidate, preference in preferences.items():
        if candidate != c1 and candidate != c2:
            if preference[c1] > preference[c2]:
                c1_votes+=1
            elif preference[c1] == preference[c2]:
                c1_votes +=1
                c2_votes += 1
            else:
                c2_votes +=1

    return c1_votes, c2_votes



def sequential_voting(preferences):
    # the number of voting competition is going to be the number of agents - 1 = nr of preferences
    # choose two agents to compete
    # the agents that compete do not vote
    assert len(preferences) >= 2

    c1 = random.randint(0, len(preferences) - 1)
    c2 = random.randint(0 , len(preferences) - 1)

    while c1 == c2:
        c2 = random.randint(0 , len(preferences) - 1)

    print(c1, c2)

    out_of_tournament = set()

    while len(out_of_tournament) != len(preferences) - 1:
        print("competing " , c1, c2)
        c1_votes, c2_votes = vote_counters(c1, c2, preferences)
        if c1_votes > c2_votes:
            out_of_tournament.add(c2)
            winner = c1
            if len(out_of_tournament) == len(preferences) - 1:
                break
            c2 = random.randint(0 , len(preferences) - 1)
            while c1 == c2 or c2 in out_of_tournament:
                c2 = random.randint(0 , len(preferences) - 1)
        else:
            out_of_tournament.add(c1)
            winner = c2
            if len(out_of_tournament) == len(preferences) - 1:
                break
            c1 = random.randint(0 , len(preferences) - 1)
            while c1 == c2 or c1 in out_of_tournament:
                c1 = random.randint(0 , len(preferences) - 1)

    
    return winner



print("winner", sequential_voting(preferences))


3 4
competing  3 4
competing  1 4
competing  0 4
competing  2 4
winner 4


In [677]:
#  kinda bourda count
def count_weird(preferences):
    # baseado na percentagem que cada um mete em que lugar

    # 2 > 4 > 3 > 1  
    # 4 > 2 > 0 > 3
    # 3 > 1 > 0 > 4
    # 4 > 1 > 2 > 0
    # 3 > 0 > 1 > 2
    
    # pontuation => [k - 1 => 1 lugar .... 0 => 4 lugar]
    # 2 => 1 lugar, 2 lugar, 3 lugar, 4 lugar => k - 1 + k - 2 + k - 3 + 0
    # 1 => 4 lugar, 2 lugar, 2 lugar, 4 lugar
    # 0 => 3 lugar, 3 lugar, 3 lugar, 2 lugar
    # 3 => 1 lugar, 1 lugar, 3 lugar, 4 lugar
    # 4 => 2 lugar, 1 lugar, 4 lugar, 1 lugar

    #  choose o que da mais

    placement_counter = defaultdict()
    # pontuation = len(preferences ) - position - 1


    for agent, preference in preferences.items():
        # sorted based on preferences
        current_preference = {k: v for k, v in sorted(preference.items(), key=lambda item: item[1])}

        pontuation_to_give = len(preference) - 1
        index = 1
        for position in current_preference.keys():
            placement_counter[position] = pontuation_to_give - index
            index-=1
    
    return placement_counter

# key => model, value => pontuation
count_weird(preferences)

defaultdict(None, {1: 2, 3: 5, 4: 5, 2: 3, 0: 4})